# 1. Import and Install Dependencies

In [ ]:
# references: https://github.com/nicknochnack/ActionDetectionforSignLanguage.git

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import urllib.request
import tarfile
import pandas as pd
pd.options.mode.chained_assignment = None
import os

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh 

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR to RGB
    image.flags.writeable = False                  # Image is not writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB to BGR
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks,   mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [110]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('OP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'i love you', 'book', 'thank you', 'bye'])

# 20 videos of data
num_sequences = 20

# Total sequences after augmentation (original + flipped + noisy)
num_augmented_sequences = num_sequences * 2 
total_sequences = num_sequences + num_augmented_sequences  

# Videos of 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [111]:
for action in actions: 
    action_path = os.path.join(DATA_PATH, action)
    os.makedirs(action_path, exist_ok=True)

    # Create subdirectories for each type of transformation
    for subfolder in ['original', 'flipped', 'noisy','shifted', 'scaled', 'dropout']:
        os.makedirs(os.path.join(action_path, subfolder), exist_ok=True)
        
      # Get existing sequence folders and find the highest folder number
    files = os.listdir(action_path)
    dirmax = max([int(f) for f in files if f.isdigit()], default=0)
    
    # Loop to create new sequence folders (for both original and augmented sequences)
    for sequence in range(start_folder,  num_sequences + 1):
        sequence_folder = str(dirmax + sequence)
        sequence_folder_path = os.path.join(action_path, sequence_folder)
        
        # Create sequence folder and subdirectories
        os.makedirs(sequence_folder_path, exist_ok=True)
        
        # Create subdirectories under each sequence folder (if not already created)
        for subfolder in ['original', 'flipped', 'noisy','shifted', 'scaled', 'dropout']:
            os.makedirs(os.path.join(sequence_folder_path, subfolder), exist_ok=True)

print("Directories created successfully.")

Directories created successfully.


# 5. Collect Keypoint Values for Training and Testing

In [112]:
VID_PATH = os.path.join('data')

In [113]:
#cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(1, 1+num_sequences):
            
            video_file = os.path.join(VID_PATH, action, f"{sequence}.mp4")
            if not os.path.exists(video_file):
                print(f"Skipping {video_file}, file not found.")
                continue

            cap = cv2.VideoCapture(video_file)
            
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                # Read feed
                ret, frame = cap.read()
                if not ret:
                    print(f"Warning: Missing frame {frame_num} in {video_file}")
                    break

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Display collection status
                text = f"Collecting frames for {action}, Video {sequence}"
                cv2.putText(image, text, (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 225), 1, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)
                

                '''
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                '''
    
                
                # Original keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), 'original', f"{frame_num}.npy")
                np.save(npy_path, keypoints)

                # Data Augmentation: Horizontal Flip
                flipped_keypoints = keypoints.copy()
                flipped_keypoints[::3] *= -1  # Flip x-coordinates
                flipped_npy_path = os.path.join(DATA_PATH, action, str(sequence), 'flipped', f"{frame_num}.npy")
                np.save(flipped_npy_path, flipped_keypoints)

                # Data Augmentation: Add Noise
                noise = np.random.normal(0, 0.01, keypoints.shape)
                noisy_keypoints = keypoints + noise
                noisy_npy_path = os.path.join(DATA_PATH, action, str(sequence), 'noisy', f"{frame_num}.npy")
                np.save(noisy_npy_path, noisy_keypoints)

                # Shift x and y coords (x: 0, 3, 6,... ; y: 1, 4, 7,...)
                shifted_keypoints = keypoints.copy()
                shifted_keypoints[::3] += 0.02  # shift x
                shifted_keypoints[1::3] += 0.02  # shift y
                shifted_npy_path = os.path.join(DATA_PATH, action, str(sequence), 'shifted', f"{frame_num}.npy")
                np.save(shifted_npy_path, shifted_keypoints)

                scaled_keypoints = keypoints * 1.1
                scaled_npy_path = os.path.join(DATA_PATH, action, str(sequence), 'scaled', f"{frame_num}.npy")
                np.save(scaled_npy_path, scaled_keypoints)

                dropout_keypoints = keypoints.copy()
                mask = np.random.rand(*keypoints.shape) < 0.1  # 10% chance to drop
                dropout_keypoints[mask] = 0
                dropout_npy_path = os.path.join(DATA_PATH, action, str(sequence), 'dropout', f"{frame_num}.npy")
                np.save(dropout_npy_path, dropout_keypoints)

                # Break using q
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()
print("Data collection and augmentation complete.")

Data collection and augmentation complete.


# 6. Preprocess Data and Create Labels and Features

In [114]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [115]:
label_map = {label:num for num, label in enumerate(actions)}

In [116]:
label_map

{'hello': 0, 'i love you': 1, 'book': 2, 'thank you': 3, 'bye': 4}

In [117]:
sequences, labels = [], []
augmentations = ['original', 'flipped', 'noisy', 'shifted', 'scaled', 'dropout']

for action in actions:
    for sequence in range(1, num_sequences + 1):
        for aug in augmentations:
            window = []
            for frame_num in range(sequence_length):
                frame_path = os.path.join(DATA_PATH, action, str(sequence), aug, f"{frame_num}.npy")
                if not os.path.exists(frame_path):
                    print(f"Missing frame: {frame_path}")
                    break
                res = np.load(frame_path)
                window.append(res)

            if len(window) == sequence_length:
                sequences.append(window)
                labels.append(label_map[action])


In [118]:
np.array(sequences).shape

(600, 30, 1662)

In [119]:
np.array(labels).shape

(600,)

In [120]:
X = np.array(sequences)

In [121]:
print(len(sequences))

600


In [122]:
X.shape

(600, 30, 1662)

In [123]:
y = to_categorical(labels).astype(int)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [125]:
y_test.shape

(120, 5)

# 7. Build and Train LSTM Neural Network

In [126]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
import tensorflow as tf

In [127]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [128]:
model = Sequential([
Input(shape=(30,1662)),
LSTM(64, return_sequences=True, activation='tanh'),
LSTM(128, return_sequences=True, activation='tanh'),
LSTM(64, return_sequences=False, activation='tanh'),
Dense(64, activation='relu'),
Dense(32, activation='relu'),
Dense(actions.shape[0], activation='softmax'),
])

In [129]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [130]:
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

In [131]:
model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test), callbacks=[tb_callback, early_stopping])

Epoch 1/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 113ms/step - accuracy: 0.1811 - loss: 1.6095 - val_accuracy: 0.1833 - val_loss: 1.5992
Epoch 2/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.2626 - loss: 1.5751 - val_accuracy: 0.2917 - val_loss: 1.5477
Epoch 3/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.3147 - loss: 1.5072 - val_accuracy: 0.2083 - val_loss: 1.6578
Epoch 4/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.2346 - loss: 1.5809 - val_accuracy: 0.2000 - val_loss: 1.5744
Epoch 5/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.3105 - loss: 1.5085 - val_accuracy: 0.3833 - val_loss: 1.4969
Epoch 6/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.3562 - loss: 1.4429 - val_accuracy: 0.3750 - val_loss: 1.4278
Epoch 7/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.4252 - loss: 1.3540 - val_accuracy: 0.3583 - val_loss: 1.4459
Epoch 8/1000
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.4225 - loss: 1.4095 - val_acc

In [132]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                     ┃ Output Shape             ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                    │ (None, 30, 64)           │       442,112 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                    │ (None, 30, 128)          │        98,816 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                    │ (None, 64)               │        49,408 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense_6 (Dense)                  │ (None, 64)               │         4,160 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense_7 (Dense)                  │ (None, 32)               │         2,080 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense_8 (Dense)                  │ (None, 5)                │           165 │
└──────────────────────────────────┴──────────────────────────┴───────────────┘

 Total params: 1,790,225 (6.83 MB)

 Trainable params: 596,741 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,193,484 (4.55 MB)

# 8. Make Predictions

In [133]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

In [134]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9071 - loss: 0.2218
Test Loss: 0.24197062849998474
Test Accuracy: 0.9083333611488342


In [135]:
print("Predicted labels:", y_pred_classes)
print("True labels:", y_true_classes)

Predicted labels: [0 4 0 0 0 0 0 2 3 0 0 3 2 0 0 2 2 0 0 0 0 0 0 0 2 2 0 4 2 2 0 0 2 2 0 0 2
 0 2 0 0 0 0 4 0 2 0 3 0 2 0 2 4 0 0 2 2 2 1 2 3 0 2 3 3 0 2 3 2 4 0 0 2 0
 0 0 0 4 0 2 4 3 4 0 3 0 0 0 4 0 2 4 2 0 4 0 0 0 2 0 2 2 4 4 2 0 0 4 0 3 2
 0 0 0 4 0 0 1 0 0]
True labels: [1 4 1 3 0 1 1 2 0 1 0 4 2 3 0 2 2 3 1 3 0 0 1 0 2 2 1 4 2 2 0 1 2 2 0 1 2
 3 2 1 1 0 3 4 1 2 1 3 3 2 0 2 4 1 0 2 2 2 3 2 4 0 2 4 4 1 2 4 2 4 3 0 2 1
 1 3 3 4 1 2 4 0 2 1 4 0 1 1 4 1 2 2 2 1 2 1 0 1 2 1 2 2 4 2 2 3 1 4 1 4 2
 0 0 0 4 1 0 3 0 0]


In [136]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()


multilabel_confusion_matrix(ytrue, yhat)


accuracy_score(ytrue, yhat)

4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 172ms/step


0.9083333333333333

# 9. Save Weights

In [137]:
# Save the model to a file
model.save('sign_language_model_2.keras')

In [ ]:
#skip if just trained
model = tf.keras.models.load_model("sign_language_model_13.keras")

# 10. Test in Real Time

In [138]:
from scipy import stats
import cv2
import numpy as np
import torch
import torchvision.models as models

In [139]:
def prob_viz2(res, actions, input_frame):
    # Find the index of the highest probability
    max_prob_index = np.argmax(res)
    max_prob_value = res[max_prob_index]
    sign = actions[max_prob_index]
    
    # Convert probability to 2 decimal places
    prob_value = f"{max_prob_value:.2f}"
    
    # Return the sign and its probability value
    return sign, prob_value

In [140]:

colors = [(245,117,16), (117,245,16), (16,117,245), (16,117,245), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [141]:

plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

IndexError: list index out of range

<Figure size 1800x1800 with 0 Axes>

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [107]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5
# Prompt the user for the video file path
video_file = input("Please enter the path to the video file (without extension): ")
input_folder = "input_data"

# Create the full video file path by appending the extension
video_path = os.path.join(input_folder, f"{video_file}.mp4")

# Check if the video file exists
if not os.path.exists(video_path):
    print(f"Error: Video file '{video_filename}.mp4' not found in the '{input_folder}' folder.")
    exit()

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Failed to open video file at {video_path}")
    exit()
    
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Check if video has frames to read
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"Video contains {frame_count} frames.")
        
        # Read feed
        ret, frame = cap.read()
        if not ret:
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Extract keypoints and append to sequence
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Keep only the latest 30 frames

        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res)

            # Display the predicted action
            predicted_action, prob_value = prob_viz2(res, actions, frame)
            
            # Output the result to the console
            print(f"Predicted sign: {predicted_action}, Probability: {prob_value}")
            
            # Return predicted sign and probability (for function)
            # return predicted_sign, prob_value

        # Viz probabilities
        image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        # Break with q
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

Please enter the path to the video file (without extension):  hello


Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
Video contains 36 frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[9.2701757e-01 5.3666015e-03 1.9687294e-09 1.3680004e-05 6.7602232e-02]
Predicted sign: hello, Probability: 0.93
Video contains 36 frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[8.80

# 11. Convert Model to .tflite file for android use

In [ ]:
model = tf.keras.models.load_model('sign_language_model2.keras')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
                                          tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [ ]:
 # Saving the model.
mpath ='sign_language_model2.tflite'
with open(mpath, 'wb') as f:
    f.write(tflite_model)